<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Question1-Knowledge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import io
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd 
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
from collections import Counter
from pprint import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("pubmed_results.csv")
df.dropna(inplace=True)


In [5]:
def extract_words(text):
    temp = text.split() # Split the text on whitespace
    text_words = []

    for word in temp:
        # Remove any punctuation characters present in the beginning of the word
        while word[0] in string.punctuation:
            word = word[1:]

        # Remove any punctuation characters present in the end of the word
        while word[-1] in string.punctuation:
            word = word[:-1]

        # Append this word into our list of words.
        text_words.append(word.lower())
        
    return text_words

In [6]:
all_documents = []
all_sentences = df['title'].to_numpy()

# for line in all_sentences:
#   all_words.extend(line.strip().split())
all_documents = [nltk.word_tokenize(sent) for sent in all_sentences]

In [8]:
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_documents[i] = [w for w in all_documents[i] if w not in stopwords.words('english')]

In [9]:
def tf_idf(corpus_words):
    IDF = inv_doc_freq(corpus_words)
    
    TFIDF = []
    
    for document in corpus_words:
        TFIDF.append(Counter(document))
    
    for document in TFIDF:
        for word in document:
            document[word] = document[word]*IDF[word]
            
    return TFIDF

def inv_doc_freq(corpus_words):
    number_docs = len(corpus_words)
    
    document_count = {}

    for document in corpus_words:
        word_set = set(document)

        for word in word_set:
            document_count[word] = document_count.get(word, 0) + 1
    
    IDF = {}
    
    for word in document_count:
        IDF[word] = np.log(number_docs/document_count[word])
        
    
    return IDF

def term_document_matrix(TFIDF, word_list, word_dict):
    vocabulary_size = len(word_dict)
    number_documents = len(TFIDF)
    
    TDM = np.zeros((vocabulary_size, number_documents))
    
    for doc in range(number_documents):
        document = TFIDF[doc]
        
        for word in document.keys():
            pos = word_dict[word]
            
            TDM[pos, doc] = document[word]
            
    return TDM

def build_vocabulary(TFIDF):
    words = set()
    
    for document in TFIDF:
        words |= document.keys()
    
    word_list = list(words)
    word_dict = dict(zip(word_list, range(len(word_list))))
    
    return word_dict, word_list

In [ ]:
IDF = inv_doc_freq(all_documents)

pprint(IDF)

In [ ]:
IDF


In [15]:
TFIDF = tf_idf(all_documents)

In [16]:
word_dict, word_list = build_vocabulary(TFIDF)

In [17]:
TDM = term_document_matrix(TFIDF, word_list, word_dict)

In [19]:
TDM.shape

(4946, 808)

In [20]:
def find_related_docs(tweet, TDM):
    new_vector = np.zeros(TDM.shape[1])
    
    for word in tweet:
        if word in word_dict:
          pos = word_dict[word]
          new_vector += TDM[pos, :]
        
    # Now the entries of new_vector tell us which documents are activated by this one.
    # Let's extract the list of documents sorted by activation
    doc_list = sorted(zip(range(TDM.shape[1]), new_vector), key=lambda x:x[1], reverse=True)
    
    return doc_list

In [ ]:
research_words = ["Obesity", "Cancer", "Covid-19", "wearable", "mental health", "influenza"]
related = find_related_docs(research_words, TDM)
related

In [27]:
for tweet, score in related[:14]:
    print(tweet,score, " ".join(all_documents[tweet]))

167 6.694562058521095 Clinical Specificities Obesity Care : The Transformations Dissolution 'Will ' 'Drives ' .
803 6.694562058521095 Using Whatsapp Consultation Covid-19 Suspected Patients Pandemic Polyclinics ; A Retrospective Case Control Study .
402 6.0014148779611505 Enhanced virulence clade 2.3.2.1 highly pathogenic avian influenza A H5N1 viruses ferrets .
567 6.0014148779611505 Impact variations potential glycosylation sites hemagglutinin H9N2 influenza virus .
340 4.497337481184876 Single-Site Robotic Radical Hysterectomy Sentinel Lymphnode Biopsy Cervical Cancer : A Case Report .
546 4.497337481184876 Lung cancer smoking trends young Switzerland : study based data National Institute Cancer Epidemiology Registration Swiss Health Surveys .
573 4.497337481184876 Staging prognosis oropharyngeal carcinoma according 8th Edition American Joint Committee Cancer Staging Manual human papillomavirus infection .
589 4.497337481184876 lincRNA-p21 Mediates Anti-Cancer Effect Ginkgo Biloba E

In [ ]:

 
rake_object = RAKE.rake('SmartStoplist.txt', 5, 3, 4)

TypeError: ignored